In [1]:
import pandas as pd
import numpy as np
import sys, os, shutil

In [2]:
df = pd.read_csv('hotel_bookings.csv')
print(df.shape)
df

(119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [3]:
df['booking_id'] = range(len(df))
df['arrival_date'] = pd.to_datetime(df['arrival_date_year'].astype(str) + '-' +
                                        df['arrival_date_month'] + '-' +
                                        df['arrival_date_day_of_month'].astype(str), format='%Y-%B-%d')
df['booking_date'] = (df['arrival_date'] - pd.to_timedelta(df['lead_time'], unit='d')).dt.date
df['booking_date'] = pd.to_datetime(df['booking_date'])
df = df[['booking_id'] + [col for col in df.columns if col != 'booking_id']]
print(df.shape)
df.head()

(119390, 35)


,booking_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,booking_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2014-07-24
1,1,Resort Hotel,0,737,2015,July,27,1,0,0,...,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2013-06-24
2,2,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-06-24
3,3,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-06-18
4,4,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-06-17


In [4]:
df['booking_date'].min()

Timestamp('2013-06-24 00:00:00')

In [5]:
# maximum date, so we will use the month of August for day-by-day daily ingestion simulation
df['booking_date'].max()

Timestamp('2017-08-31 00:00:00')

In [6]:
df_training = df[df['booking_date'] <= '2017-07-31']
print(df_training.shape)
df_training.head()

(118660, 35)


,booking_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,booking_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2014-07-24
1,1,Resort Hotel,0,737,2015,July,27,1,0,0,...,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,2013-06-24
2,2,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-06-24
3,3,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,2015-06-18
4,4,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2015-06-17


In [7]:
df_prediction = df[df['booking_date'] >= '2017-08-01']
print(df_prediction.shape)
df_prediction.head()

(730, 35)


,booking_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,booking_date
13222,13222,Resort Hotel,1,3,2017,August,31,5,2,1,...,NaN,0,Transient,230.0,0,1,Canceled,2017-08-03,2017-08-05,2017-08-02
13326,13326,Resort Hotel,1,0,2017,August,32,10,0,1,...,NaN,0,Transient,8.0,0,0,Canceled,2017-08-10,2017-08-10,2017-08-10
13381,13381,Resort Hotel,1,0,2017,August,33,13,1,0,...,NaN,0,Transient,207.0,0,0,No-Show,2017-08-13,2017-08-13,2017-08-13
13402,13402,Resort Hotel,1,0,2017,August,33,14,1,0,...,NaN,0,Transient,225.0,0,0,Canceled,2017-08-14,2017-08-14,2017-08-14
13403,13403,Resort Hotel,1,0,2017,August,33,14,1,0,...,NaN,0,Transient,230.0,0,0,Canceled,2017-08-14,2017-08-14,2017-08-14


In [8]:
df_training.to_csv('hotel_bookings_training.csv', index=False)
df_prediction.to_csv('hotel_bookings_prediction.csv', index=False)